In [1]:
from pathlib import Path
import requests
import zipfile
import numpy as np
import pandas as pd
import csv

data_path = Path.cwd().parent / "data"
fips_csv_path = data_path / "fips_states_2010.csv"
csv_path = data_path / "dataset" / "ejscreen_2020"

In [2]:
download = requests.get("https://gaftp.epa.gov/EJSCREEN/2020/EJSCREEN_2020_StatePctile.csv.zip", verify=False)
file_contents = download.content
zip_file_path = data_path / "tmp"
zip_file = open(zip_file_path / "downloaded.zip", "wb")
zip_file.write(file_contents)
zip_file.close()

/Users/lucas/.virtualenvs/scoring/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gaftp.epa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [3]:
with zipfile.ZipFile(zip_file_path / "downloaded.zip", "r") as zip_ref:
    zip_ref.extractall(zip_file_path)
ejscreen_csv = data_path / "tmp" / "EJSCREEN_2020_StatePctile.csv"

In [4]:
df = pd.read_csv(ejscreen_csv, dtype={'ID': 'string'}, low_memory=False)

In [5]:
# write nationwide csv
df.to_csv(csv_path / f"usa.csv", index = False)

In [6]:
# write per state csvs
with open(fips_csv_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=",")
    line_count = 0

    for row in csv_reader:
        if line_count == 0:
            line_count += 1
        else:
            fips = row[0].strip()
            print(f"Generating data{fips} csv")
            df1 = df[df.ID.str[:2] == fips]
            # we need to name the file data01.csv for ogr2ogr csv merge to work
            df1.to_csv(csv_path / f"data{fips}.csv", index = False)

Generating data01 csv
Generating data02 csv
Generating data04 csv
Generating data05 csv
Generating data06 csv
Generating data08 csv
Generating data09 csv
Generating data10 csv
Generating data11 csv
Generating data12 csv
Generating data13 csv
Generating data15 csv
Generating data16 csv
Generating data17 csv
Generating data18 csv
Generating data19 csv
Generating data20 csv
Generating data21 csv
Generating data22 csv
Generating data23 csv
Generating data24 csv
Generating data25 csv
Generating data26 csv
Generating data27 csv
Generating data28 csv
Generating data29 csv
Generating data30 csv
Generating data31 csv
Generating data32 csv
Generating data33 csv
Generating data34 csv
Generating data35 csv
Generating data36 csv
Generating data37 csv
Generating data38 csv
Generating data39 csv
Generating data40 csv
Generating data41 csv
Generating data42 csv
Generating data44 csv
Generating data45 csv
Generating data46 csv
Generating data47 csv
Generating data48 csv
Generating data49 csv
Generating